# HP Processing Pipeline - DOD Template Based

Process radar files and output to DOD-compliant NetCDF using template header structure.

**Workflow:**
0. Make netcdf from DOD
1. Provide DOD template NetCDF file path
2. Provide radar input file path  
3. Process radar data (classify, grid)
4. Write output using DOD template header with processed data

In [2]:
# Import Required Libraries
import netCDF4
import numpy as np
import xarray as xr
from pathlib import Path
import sys
import act

# Add path to access hp_processing functions
sys.path.insert(0, '/Users/bhupendra/projects/sail/Python')

print("Libraries loaded successfully")

Libraries loaded successfully


## 0. Create NetCDF From DOD

In [ ]:
# Define DOD parameters for xprecipradarhp-c1-1.3
proc = 'xprecipradarhp.c1'
version = '1.3'

# Set dimensions as per DOD spec
set_dims = {
    'time': 1,  # UNLIMITED, but we process one time slice
    'y': 160,
    'x': 160,
    'bound': 2
}

DOD_FILL_VALUE = -9999.0
scalar_fill_dim = 'time'  # For scalar variables with time dimension

print(f"Creating DOD dataset: {proc} {level} v{version}")
print(f"Dimensions: {set_dims}")

dod_ds = act.io.create_ds_from_arm_dod(
    proc, 
    set_dims, 
    version=version,
    fill_value=DOD_FILL_VALUE,
    scalar_fill_dim=scalar_fill_dim,
    local_file=False
)


Creating DOD dataset: xprecipradarhp.c1 c1 v1.3
Dimensions: {'time': 1, 'y': 160, 'x': 160, 'bound': 2}
✓ DOD dataset created successfully
  Dimensions: {'time': 1, 'y': 160, 'x': 160}
  Variables: ['base_time', 'time_offset', 'corrected_reflectivity', 'hp_fhc', 'hp_ssc', 'lowest_height', 'lat', 'lon', 'radar_lat', 'radar_lon', 'radar_alt']
  Coordinates: ['time', 'y', 'x']
✓ DOD dataset created successfully
  Dimensions: {'time': 1, 'y': 160, 'x': 160}
  Variables: ['base_time', 'time_offset', 'corrected_reflectivity', 'hp_fhc', 'hp_ssc', 'lowest_height', 'lat', 'lon', 'radar_lat', 'radar_lon', 'radar_alt']
  Coordinates: ['time', 'y', 'x']


/var/folders/c0/hb5cyy892hqdjht868lw3qk80000gp/T/ipykernel_10026/555310920.py:29: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Dimensions: {dict(dod_ds.dims)}")


In [16]:
# Save DOD template to compressed NetCDF4 file
dod_file_path = '/Users/bhupendra/projects/sail/vaps/hp_proc/dod_template.nc'

# Convert xarray to NetCDF4 format with compression
encoding = {}
for var in dod_ds.data_vars:
    encoding[var] = {
        'zlib': True,
        'complevel': 4,
        'dtype': dod_ds[var].dtype
    }

dod_ds.to_netcdf(dod_file_path, encoding=encoding, format='NETCDF4')
print(f"✓ DOD template saved: {dod_file_path}")
print(f"  File size: {Path(dod_file_path).stat().st_size / 1024:.1f} KB")
print(f"  Variables: {list(dod_ds.data_vars.keys())}")
print(f"  Dimensions: {dict(dod_ds.dims)}")

✓ DOD template saved: /Users/bhupendra/projects/sail/vaps/hp_proc/dod_template.nc
  File size: 68.0 KB
  Variables: ['base_time', 'time_offset', 'corrected_reflectivity', 'hp_fhc', 'hp_ssc', 'lowest_height', 'lat', 'lon', 'radar_lat', 'radar_lon', 'radar_alt']
  Dimensions: {'time': 1, 'y': 160, 'x': 160}


/var/folders/c0/hb5cyy892hqdjht868lw3qk80000gp/T/ipykernel_10026/3442009200.py:17: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Dimensions: {dict(dod_ds.dims)}")


## 1. Define Function to Read NetCDF Header

Extract structure (dimensions, variables, attributes) from DOD template file.

In [12]:
def read_netcdf_header(dod_file_path):
    """
    Extract header information from DOD template NetCDF file.
    
    Returns dict with:
    - dimensions: {name: size}
    - variables: {name: (dims, dtype, attributes)}
    - global_attrs: {name: value}
    """
    with netCDF4.Dataset(dod_file_path, 'r') as ds:
        header = {
            'dimensions': dict(ds.dimensions),
            'variables': {},
            'global_attrs': {k: ds.getncattr(k) for k in ds.ncattrs()}
        }
        
        for var_name in ds.variables:
            var = ds.variables[var_name]
            header['variables'][var_name] = {
                'dims': var.dimensions,
                'dtype': var.dtype,
                'shape': var.shape,
                'attrs': {k: var.getncattr(k) for k in var.ncattrs()}
            }
    
    return header


## 2. Define Function to Create Output NetCDF with Header

Create output file with same structure as DOD template (empty, ready for data).

In [13]:
def create_output_netcdf(output_path, header):
    """
    Create output NetCDF file with DOD template structure.
    Returns open Dataset for writing data.
    """
    ds = netCDF4.Dataset(output_path, 'w', format='NETCDF4')
    
    # Create dimensions
    for dim_name, dim_size in header['dimensions'].items():
        ds.createDimension(dim_name, dim_size)
    
    # Create variables with attributes
    for var_name, var_info in header['variables'].items():
        var = ds.createVariable(
            var_name,
            var_info['dtype'],
            dimensions=var_info['dims'],
            fill_value=var_info['attrs'].get('_FillValue', None)
        )
        
        # Add attributes
        for attr_name, attr_value in var_info['attrs'].items():
            if attr_name != '_FillValue':
                try:
                    var.setncattr(attr_name, attr_value)
                except Exception as e:
                    print(f"Warning: Could not set {attr_name} for {var_name}: {e}")
    
    # Add global attributes
    for attr_name, attr_value in header['global_attrs'].items():
        try:
            ds.setncattr(attr_name, attr_value)
        except Exception as e:
            print(f"Warning: Could not set global attribute {attr_name}: {e}")
    
    return ds


## 3. Define Function to Read and Process Radar File

Process radar data using existing hp_processing functions.

In [14]:
def process_radar_file(radar_file, season='summer'):
    """
    Process radar file and return processed data dict with:
    - corrected_reflectivity: (time, y, x)
    - hp_fhc: (time, y, x)
    - hp_ssc: (time, y, x)
    - lowest_height: (time, y, x)
    - lat: (y, x)
    - lon: (y, x)
    - radar_lat, radar_lon, radar_alt: scalars
    """
    from hp_processing import process_radar, make_squire_grid
    
    print(f"Reading {radar_file}...")
    radar = process_radar(radar_file, season)
    
    print("Gridding and extracting data...")
    data_ds = make_squire_grid(radar)
    
    # Extract lowest vertical level
    dbz = data_ds['corrected_reflectivity'].values[0, :, :]  # (y, x)
    hp_fhc = data_ds['hp_fhc'].values[0, :, :] if 'hp_fhc' in data_ds else np.full((160, 160), -9999, dtype=np.int16)
    hp_ssc = data_ds['hp_ssc'].values[0, :, :] if 'hp_ssc' in data_ds else np.full((160, 160), -9999, dtype=np.int16)
    lowest_h = data_ds['lowest_height'].values[0, :, :] if 'lowest_height' in data_ds else np.full((160, 160), -9999)
    lat = data_ds['lat'].values  # (y, x)
    lon = data_ds['lon'].values  # (y, x)
    
    # Expand to time dimension (1, y, x)
    dbz = np.expand_dims(dbz, axis=0)
    hp_fhc = np.expand_dims(hp_fhc, axis=0)
    hp_ssc = np.expand_dims(hp_ssc, axis=0)
    lowest_h = np.expand_dims(lowest_h, axis=0)
    
    return {
        'corrected_reflectivity': dbz,
        'hp_fhc': hp_fhc,
        'hp_ssc': hp_ssc,
        'lowest_height': lowest_h,
        'lat': lat,
        'lon': lon,
        'radar_lat': float(data_ds.attrs.get('radar_latitude', -9999)),
        'radar_lon': float(data_ds.attrs.get('radar_longitude', -9999)),
        'radar_alt': float(data_ds.attrs.get('radar_altitude', -9999)),
    }



## 4. Define Function to Write Processed Data to Output

Populate output NetCDF with processed radar data.

In [15]:
def write_data_to_netcdf(ds, processed_data):
    """
    Write processed radar data to NetCDF Dataset.
    """
    try:
        # Write main data variables
        if 'corrected_reflectivity' in ds.variables:
            ds.variables['corrected_reflectivity'][:] = processed_data['corrected_reflectivity']
        
        if 'hp_fhc' in ds.variables:
            ds.variables['hp_fhc'][:] = processed_data['hp_fhc'].astype(ds.variables['hp_fhc'].dtype)
        
        if 'hp_ssc' in ds.variables:
            ds.variables['hp_ssc'][:] = processed_data['hp_ssc'].astype(ds.variables['hp_ssc'].dtype)
        
        if 'lowest_height' in ds.variables:
            ds.variables['lowest_height'][:] = processed_data['lowest_height']
        
        # Write coordinate variables
        if 'lat' in ds.variables:
            ds.variables['lat'][:] = processed_data['lat']
        
        if 'lon' in ds.variables:
            ds.variables['lon'][:] = processed_data['lon']
        
        # Write scalar radar location variables
        if 'radar_lat' in ds.variables:
            ds.variables['radar_lat'][0] = processed_data['radar_lat']
        
        if 'radar_lon' in ds.variables:
            ds.variables['radar_lon'][0] = processed_data['radar_lon']
        
        if 'radar_alt' in ds.variables:
            ds.variables['radar_alt'][0] = processed_data['radar_alt']
        
        print("Data written successfully")
        
    except Exception as e:
        print(f"Error writing data: {e}")
        raise


## 5. MAIN EXECUTION - Provide Your File Paths Here

Edit these three variables with your actual file paths:

In [ ]:
# EDIT THESE THREE VARIABLES
dod_template_file = "/Users/bhupendra/projects/sail/vaps/hp_proc/dod_template.nc"  # DOD template NetCDF file
radar_input_file = "/"     # Radar input file
output_file = "/path/to/output.nc"              # Output file path
season = "summer"                                # "summer" or "winter"

print(f"DOD Template: {dod_template_file}")
print(f"Radar Input: {radar_input_file}")
print(f"Output File: {output_file}")
print(f"Season: {season}")

### Step 1: Read DOD Template Header

In [ ]:
header = read_netcdf_header(dod_template_file)
print(f"✓ Header loaded from {Path(dod_template_file).name}")
print(f"  Dimensions: {dict(header['dimensions'])}")
print(f"  Variables: {list(header['variables'].keys())}")

### Step 2: Process Radar File

In [ ]:
processed_data = process_radar_file(radar_input_file, season)
print(f"✓ Radar processing complete")
print(f"  corrected_reflectivity shape: {processed_data['corrected_reflectivity'].shape}")
print(f"  hp_fhc shape: {processed_data['hp_fhc'].shape}")
print(f"  hp_ssc shape: {processed_data['hp_ssc'].shape}")
print(f"  lowest_height shape: {processed_data['lowest_height'].shape}")
print(f"  lat shape: {processed_data['lat'].shape}")
print(f"  radar location: ({processed_data['radar_lat']}, {processed_data['radar_lon']}, {processed_data['radar_alt']}m)")

### Step 3: Create Output NetCDF with DOD Header

In [ ]:
output_ds = create_output_netcdf(output_file, header)
print(f"✓ Output NetCDF created: {output_file}")
print(f"  Dimensions: {dict(output_ds.dimensions)}")
print(f"  Variables: {list(output_ds.variables.keys())}")

### Step 4: Write Processed Data to Output

In [ ]:
write_data_to_netcdf(output_ds, processed_data)
output_ds.close()
print(f"✓ Output file closed: {output_file}")

## 6. Verify Output File

In [ ]:
print("\n" + "="*60)
print("OUTPUT FILE VERIFICATION")
print("="*60)

with netCDF4.Dataset(output_file, 'r') as verify_ds:
    print(f"\nFile: {output_file}")
    print(f"\nDimensions:")
    for dim_name, dim_size in verify_ds.dimensions.items():
        print(f"  {dim_name}: {dim_size}")
    
    print(f"\nVariables and shapes:")
    for var_name in verify_ds.variables:
        var = verify_ds.variables[var_name]
        print(f"  {var_name}: {var.shape} ({var.dtype})")
    
    print(f"\nGlobal attributes ({len(verify_ds.ncattrs())}):")
    for attr in list(verify_ds.ncattrs())[:5]:  # Show first 5
        print(f"  {attr}")
    
    print("\n✓ Output file structure verified")